In [13]:
#https://di-acc2.com/programming/python/21858/#index_id9
#market_making_MEXC_Bot オリジナル
import hashlib
import hmac
import requests
import datetime
import json
import base64
import time
import urllib.parse
import random
import ccxt
from pycoingecko import CoinGeckoAPI

hurdle_ratio=0.18
orderID2=0
result1={}
Buy_Price_previous=0.0
Sell_Price_previous=0.0
#提示レート決定
market_pair = 'YOUR_market_pair'

API_KEY    = "YOUR_API_KEY"
SECRET_KEY = "YOUR_SECRET_KEY"

# MEXCのインスタンス作成
mexc = ccxt.mexc({'apiKey':API_KEY,'secret':SECRET_KEY})

#オーダー数のカウント（MEXC）ピックアップオーダーリスト
# 注文照会
trade_info = mexc.fetchOpenOrders(market_pair)

# 出力
order_fulfill_num= 52 - len(trade_info)
#while True:

if True:

    #発注--------------------------------------
    cg = CoinGeckoAPI()
    xwin=cg.get_price(ids='YOUR_id', vs_currencies=['usd', 'btc'])['YOUR_id']['usd']
    
    xwin_address='YOUR__address'
    busd_address='YOUR__address'
    sxp_address='YOUR__address'
    btcb_address='YOUR__address'

    maisu =50
    maisu =10**18*maisu
    num = str(maisu)
    pathOneinch_url="https://api.1inch.exchange/v3.0/56/quote?fromTokenAddress="
    r = requests.get(pathOneinch_url + busd_address + "&toTokenAddress=" + xwin_address + "&amount=" + num)
    j = r.json()
    Buy_Price=float(j["fromTokenAmount"])/float(j["toTokenAmount"])

    r = requests.get(pathOneinch_url+ xwin_address + "&toTokenAddress=" + busd_address + "&amount=" + num)
    j = r.json()
    Sell_Price=float(j["toTokenAmount"])/float(j["fromTokenAmount"])

    if xwin*(1-hurdle_ratio) <= Buy_Price < xwin*(1+hurdle_ratio)and Sell_Price < Buy_Price*(1+hurdle_ratio*.1) :
        Buy_Price=round(Buy_Price,4)
        Buy_Price_previous=Buy_Price
    else:
        Buy_Price=Buy_Price_previous
    #1inchの買いオーダー
    Buy_Price_text=str(Buy_Price)

    if xwin*(1-hurdle_ratio) <= Sell_Price < xwin*(1+hurdle_ratio) and Sell_Price < Buy_Price*(1+hurdle_ratio*.1) :
        Sell_Price=round(Sell_Price,4)
        Sell_Price_previous=Sell_Price
    else:
        Sell_Price=Sell_Price_previous
    #1inchの売りオーダー
    Sell_Price_text=str(Sell_Price)
    
    #基準価格の決定
    SpreadMean=(Buy_Price+Sell_Price)/2
    Buy_Base_Rate=max(Buy_Price,xwin,SpreadMean)
    Sell_Base_Rate=min(Sell_Price,xwin,SpreadMean)

    path_url = "/api/v2/order/new"
    fillordersID2=[]
    
    half_spread1=0.0005  #basis
    price_skew=0.0006  #絶対額
    scale_factor=0.01  #basis
    
    crypto_ask=max(Buy_Base_Rate,SpreadMean*(1+half_spread1))+price_skew
    crypto_bid=min(Sell_Base_Rate,SpreadMean*(1-half_spread1))+price_skew

    for counter in range(0,int(order_fulfill_num*0.5)):
        crypto_ask=crypto_ask*(1+scale_factor)
        crypto_bid=crypto_bid*(1-scale_factor)
        amount=str(round(random.uniform(50,127.5),2)) #'0.1'
        if counter==0:
             amount=str(round(random.uniform(30.61,220.98),2))
        #①発注の内容（ボディ）--------------------------------------
        crypto_ask = "{:.4f}".format(crypto_ask)
        order_string=str(crypto_ask)
        crypto_ask = float(crypto_ask)

        order = mexc.createLimitOrder(
                        symbol = market_pair,   # 取引通貨
                        price  = order_string,  # 指値価格
                        side   = 'sell',        # 購入(buy) or 売却(sell)
                        amount = amount,        # 購入数量[XWIN]
                        )

        #①発注の内容（ボディ）--------------------------------------
        crypto_bid = "{:.4f}".format(crypto_bid)
        order_string=str(crypto_bid)
        crypto_bid = float(crypto_bid)
        
        order = mexc.createLimitOrder(
                        symbol = market_pair,   # 取引通貨
                        price  = order_string,  # 指値価格
                        side   = 'buy',        # 購入(buy) or 売却(sell)
                        amount = amount,        # 購入数量[XWIN]
                        )

#--------------------------------------
    time.sleep(5)

#オーダーキャンセル（P2PB2B）ピックアップオーダーリスト
    
    if len(trade_info)==50:
        trade_info = mexc.fetchOpenOrders(market_pair)
        for num in range(0, len(trade_info)-10):
            cancel_order = mexc.cancel_order(symbol = market_pair,  # 取引通貨
                                     id = trade_info[num]['id'],    # 注文ID
                                    )